In [1]:
# Importing packages and the Keras libraries and we need to create CNN
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

% matplotlib inline

/Users/julian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Step 1 - The Sequential model is a linear stack of layers.
model = Sequential()
# Step 2 - Convolution2D creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs. 
# If  use_bias is True, a bias vector is created and added to the outputs. 
# Finally, if activation is not None, it is applied to the outputs as well.
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu', input_shape=(64, 64, 3)))
# Step 3 - Max pooling is a sample-based discretization process. 
#The objective is to down-sample an input representation (image, hidden-layer output matrix, etc.), 
#reducing its dimensionality and allowing for assumptions to be made about features contained in the sub-regions binned.
model.add(MaxPooling2D(pool_size=2))
# Step 4 - Dropout consists in randomly setting a fraction rate of input units to 0 at each update during training time, 
# which helps prevent overfitting.
model.add(Dropout(0.3))
# Step 5 - Convolution2D
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
# Step 6 - MaxPooling2D
model.add(MaxPooling2D(pool_size=2))
# Step 7 - Dropout
model.add(Dropout(0.3))
# Step 8 - Flattens the input. Does not affect the batch size.
model.add(Flatten())
# Step 9 - Dense implements the operation: output = activation(dot(input, kernel) + bias) 
# where activation is the element-wise activation function passed as the activation argument, 
# kernel is a weights matrix created by the layer, and bias is a bias vector created by the layer.
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.05))
# Step 10 - Softmax is a generalization of the logistic function that "squashes" a K-dimensional vector Z
# of arbitrary real values to a K-dimensional vector Sigma(z) of real values, 
# where each entry is in the range (0, 1), and all the entries add up to 1.
model.add(Dense(3, activation='softmax'))

In [3]:
# Step 11 - Display summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        4128      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
__________

In [4]:
# Step 12 - Compile the trained model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [5]:
# Use app from https://github.com/hardikvasa/google-images-download
# Install -> $ pip install google_images_download
#googleimagesdownload --keywords "oak leaf" --no_numbering  --limit 50 --output_directory "train" --image_directory "oakleaf"
#googleimagesdownload --keywords "olive leaf -bottle, hoja de olivo" --no_numbering  --limit 50 --output_directory "train" --image_directory "oliveleaf"
#googleimagesdownload --keywords "salix leaf" --no_numbering  --limit 50 --output_directory "train" --image_directory "salixleaf"

In [6]:
# Step 13 - Generate datagens for training set and validation set
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, vertical_flip = True, horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('data/train', target_size = (64, 64), batch_size = 32, class_mode = 'categorical', shuffle=True)
validation_set = validation_datagen.flow_from_directory('data/validation', target_size = (64, 64), batch_size = 32, class_mode = 'categorical', shuffle=True)

clazz_map = (training_set.class_indices)

Found 1009 images belonging to 3 classes.
Found 185 images belonging to 3 classes.


In [7]:
# Step 14 - Setting the checkpoint that we will can return if we started again. 
model_name = "cnn-tp3.hdf5"
checkpoint = ModelCheckpoint(model_name, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=10, verbose=1, mode='auto')
callbacks_list = [checkpoint, early_stopping]

In [12]:
# Step 15 - Train the model
model = load_model('cnn-tp3.hdf5')
model_history = model.fit_generator(
    training_set, 
    steps_per_epoch=1009/32, 
    initial_epoch=1, 
    epochs=50, 
    validation_data=validation_set, 
    validation_steps=185/32)

Epoch 2/50


/Users/julian/anaconda3/lib/python3.6/site-packages/PIL/Image.py:916: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


32/31 [==============================] - 18s 550ms/step - loss: 0.4549 - acc: 0.7841 - val_loss: 0.4075 - val_acc: 0.8306
Epoch 3/50
32/31 [==============================] - 16s 491ms/step - loss: 0.4437 - acc: 0.7715 - val_loss: 0.3916 - val_acc: 0.8324
Epoch 4/50
32/31 [==============================] - 16s 490ms/step - loss: 0.4457 - acc: 0.7887 - val_loss: 0.4020 - val_acc: 0.8288
Epoch 5/50
32/31 [==============================] - 16s 487ms/step - loss: 0.4194 - acc: 0.8007 - val_loss: 0.3805 - val_acc: 0.8234
Epoch 6/50
32/31 [==============================] - 16s 494ms/step - loss: 0.4483 - acc: 0.7791 - val_loss: 0.3890 - val_acc: 0.8396
Epoch 7/50
32/31 [==============================] - 16s 490ms/step - loss: 0.4284 - acc: 0.7828 - val_loss: 0.3996 - val_acc: 0.8270
Epoch 8/50
32/31 [==============================] - 16s 495ms/step - loss: 0.4389 - acc: 0.7892 - val_loss: 0.3497 - val_acc: 0.8523
Epoch 9/50
32/31 [==============================] - 15s 483ms/step - loss: 0.430

In [15]:
# Step 16 - Save model
model.save('cnn-tp3.hdf5')

In [16]:
# Step 17 - Prediction
prediction_image_names=['data/prediction/oakleaf1.jpg', 
                       'data/prediction/oakleaf2.jpg',
                       'data/prediction/oliveleaf1.jpg',
                       'data/prediction/oliveleaf2.jpg',
                       'data/prediction/willowleaf1.jpg',
                       'data/prediction/willowleaf2.jpg']

for image_name in prediction_image_names:
    test_image = image.load_img(image_name, target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(test_image)
    index = result.argmax(1)[0]
    
    for clazz in clazz_map.keys():
        if clazz_map[clazz] == index:
            print("The image belongs to the class: ", clazz)

The image belongs to the class:  oakleaf
The image belongs to the class:  oakleaf
The image belongs to the class:  oliveleaf
The image belongs to the class:  oliveleaf
The image belongs to the class:  oakleaf
The image belongs to the class:  oakleaf
